In [2]:
import tensorflow as tf
import numpy as np

## mnist dataset
<br>
http://yann.lecun.com/exdb/mnist/

In [3]:
import pickle


# load pickle dataset
def load(data_path):
    with open(data_path,'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [4]:
import numpy as np


class MnistData:
    def __init__(self, filenames, need_shuffle, datatype='training'):
        x_train, y_train, x_test, y_test = load(filenames) #"data/mnist.pkl"
        
        if datatype=='training':
            self._data = x_train / 127.5 -1
            self._labels = y_train
        else:
            self._data = x_test / 127.5 -1
            self._labels = y_test
        
        print(self._data.shape)
        print(self._labels.shape)
            
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

dataset = "data/mnist.pkl"
train_data = MnistData(dataset, True, 'training')
test_data = MnistData(dataset, False, 'test')

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [5]:
import tensorflow as tf
import numpy as np

# old trick again...
xs = tf.placeholder(tf.float32, [None, 28*28])
ys = tf.placeholder(tf.int64, [None])

W = tf.get_variable('w', [xs.get_shape()[-1], 10], initializer=tf.random_normal_initializer(0, 1))
b = tf.get_variable('b', [10], initializer=tf.constant_initializer(0.0))
y_ = tf.matmul(xs, W) + b

p_y = tf.nn.softmax(y_)
"""
# 5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(ys, 10, dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

loss = tf.losses.sparse_softmax_cross_entropy(labels=ys, logits=y_)

train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

init = tf.global_variables_initializer()

predict = tf.argmax(p_y, 1)
correct_prediction = tf.equal(predict, ys)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

train_steps = 100000
test_steps = 50
batch_size = 20

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_step], feed_dict={xs: batch_data, ys: batch_labels})
        if (i+1) % 1000 == 0:
            print('[Traini] Step: %d, loss: %4.5f, acc: %4.5f' %(i+1, loss_val, acc_val))
            
        if (i+1) %10000 == 0:
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = acc = sess.run(accuracy, feed_dict={xs: test_batch_data, ys: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f' % (i+1, test_acc))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[Traini] Step: 1000, loss: 0.63176, acc: 0.85000
[Traini] Step: 2000, loss: 3.87453, acc: 0.65000
[Traini] Step: 3000, loss: 0.80724, acc: 0.95000
[Traini] Step: 4000, loss: 0.38955, acc: 0.90000
[Traini] Step: 5000, loss: 1.71078, acc: 0.80000
[Traini] Step: 6000, loss: 1.78203, acc: 0.85000
[Traini] Step: 7000, loss: 1.84880, acc: 0.85000
[Traini] Step: 8000, loss: 1.31472, acc: 0.75000
[Traini] Step: 9000, loss: 1.16231, acc: 0.80000
[Traini] Step: 10000, loss: 1.10725, acc: 0.85000
[Test] Step: 10000, acc: 0.85400
[Traini] Step: 11000, loss: 1.50305, acc: 0.85000
[Traini] Step: 12000, loss: 0.55998, acc: 0.85000
[Traini] Step: 13000, loss: 1.33326, acc: 0.85000
[Traini] Step: 14000, loss: 2.41995, acc: 0.85000
[Traini] Step: 15000, loss: 0.65288, acc: 0.85000
[Traini] Step: 16000, loss: 0.21513, acc: 0.95000
[Traini] Step: 17000, loss: 1.92044, acc: 0.85000
[Train